# SparkContext și introducere în RDD

### Conținutul acestui laborator este adaptat și urmărește cartea https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf


### Importul pachetului numpy si a clasei SparkContext:


In [ ]:
from pyspark import SparkContext
import numpy as np

## Inițializarea unui `SparkContext` (abstractizarea principală către cluster)

**Observație: Valoarea '4' din argument reprezintă faptul că vor fi utilizate 4 core-uri pentru acest obiect SparkContext.**


In [ ]:
sc = SparkContext(master="local[4]")

In [ ]:
print(sc)

### Generarea unei liste de numere întregi aleatoare:


In [ ]:
lst = np.random.randint(0, 10, 20)

In [ ]:
print(lst)

### Paralelizarea listei - aceasta este operația principală către calculul distribuit:


In [ ]:
A = sc.parallelize(lst)

### Prin comanda precedentă este creat un RDD.

![](https://i.stack.imgur.com/cwrMN.png)

Spark are ca element central conceptul de Resilient Distributed Dataset (RDD). Acesta reprezintă o **colecție (rezistentă la defecte) de elemente asupra cărora se poate opera în paralel**.

SparkContext gestionează datele distribuite în nodurile _worker_ prin manager-ul de cluster.

Exista două modalități de a crea RDD-uri:

- paralelizând o colecție existentă în program (_driver program_) sau
- referind un set de date (_dataset_) dintr-un sistem de stocare extern, precum un sistem de fișiere partajat, HDFS, HBase, sau orice sursă de date ce oferă date în formatul InputFormat din Hadoop.

Crearea RDD-ului anterior a fost facută prin prima metodă.


### `A` este un obiect pyspark RDD. Nu îi putem accesa elementele direct.


In [ ]:
# Afisati tipul de date al lui A

In [ ]:
A

### Invers paralelizării - metoda `collect` colectează elementele distribuite și le returnează nodului head. <br><br>Observație: Acest proces este lent, iar din acest motiv nu trebuie utilizat frecvent.


In [ ]:
A.collect()

### Cum au fost create partițiile? Utilizați metoda `glom`.

Metoda glom() transformă fiecare partitie într-o listă de elemente. Prin urmare, se creează un RDD de liste, câte o listă pentru fiecare partiție.


In [ ]:
A.glom().collect()

### Opriți contextul și reinitializați-l cu 2 core-uri. Ce se întâmplă când repetăm procesul de mai sus?


In [ ]:
# oprire context

**RDD-ul este distribuit acum în 2 părți, în locul celor 4 anterioare.**

Recreați contextul cu 4 core-uri.


## Operații de bază

### Numărarea elementelor (`Count`):


In [ ]:
A.count()

### Obținerea primului element (`first`) și a primelor n elemente (`take`):


In [ ]:
A.first()

In [ ]:
A.take(4)

### Ștergerea duplicatelor: Obținerea unui alt RDD cu elemente distincte (`distinct`)

Metoda `RDD.distinct()` returnează un nou dataset ce conține elementele distincte ale setului de date sursă.

**Observație**: Această operație necesită o amestecare (**shuffle**) pentru a detecta duplicarea elementelor în toate partițiile. **Prin urmare, operația este lentă.**


In [ ]:
A_distinct = A.distinct()

In [ ]:
A_distinct.collect()

### Însumarea tuturor elementelor cu ajutorul metodei `reduce`

Lambda functii: Python permite crearea de funcții anonime (funcții definite fără nume), cu ajutorul construcției "lambda".
Forma generală a unei lambda funcții este: <br>
`lambda <args>: <expr>`
<br>
Lambda funcții condiționale: <br>
`lambda <args>: <expr_1> if <conditie> else <expr_2>`
<br>

Analizați documentația oficială pentru metodele RDD (în particular, reduce): https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis


In [ ]:
A.reduce(lambda x, y: x+y)

### Însumarea directă a elementelor cu ajutorul metodei `sum`:


In [ ]:
A.sum()

### Însumarea elementelor cu ajutorul metodei `fold`, care agregă elementele fiecărei partiții, iar apoi rezultatele pentru toate partițiile:


In [ ]:
A.fold(0, lambda x, y: x+y)

### Determinarea elementului maxim cu ajutorul metodei `reduce`:


In [ ]:
A.reduce(lambda x, y: x if x > y else y)

### Determinarea celui mai lung cuvânt cu ajutorul metodei `reduce`:


In [ ]:
words = 'These are some of the best Macintosh computers ever'.split(' ')
# paralelizați si obtineti rezultatul cerut aplicand reduce

## Funcții și filtrare în RDD

### Metoda `filter` returnează un RDD nou, ce conține elemente care satisfac un predicat dat (lambda expresie)


In [ ]:
# Returnează un RDD ale cărui elemente sunt divizible cu 3
A.filter(lambda x: x % 3 == 0).collect()

### Pe lângă Lambda funcții, putem crea funcții Python obișnuite pe care le vom utiliza ca argument al metodei `reduce`


In [ ]:
def largerThan(x, y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x) > len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y:
            return x
        else:
            return y

In [ ]:
wordRDD.reduce(largerThan)

## Eșantionarea (Sampling) a unui RDD

- De obicei, dimensiunea unui RDD este foarte mare.
- **Unele agregări, precum mediile aritmetice, pot fi aproximate eficient utilizând doar un eșantion de date (un sample).** Acest lucru este benefic atunci când se lucrează cu seturi de date de dimensiuni foarte mari, în care un sample poate fi relevant din punct de vedere al pattern-ului și al statisticilor descriptive ale datelor.
- Eșantionarea (sampling-ul) se realizează în paralel și necesită o capacitate de calcul limitată.

Metoda `RDD.sample(withReplacement,p)` generează un sample din elementele RDD-ului. Semnificația argumentelor metodei este următoarea:

- `withReplacement` este un flag boolean ce indică dacă un element din RDD poate fi eșantionat mai mult decât o singură dată.
- `p` reprezintă probabilitatea de acceptare a fiecărui element din sample. De remarcat că, deoarece eșantionarea se realizează independent în cadrul fiecărei partiții, numărul de elemente din eșantion se schimbă de la un eșantion la altul.


In [ ]:
# Obțineți un sample a cărui dimensiune așteptată să fie m
# Observați că dimensiunea eșantionului este diferită de la o execuție la alta
m = 5
n = 20
print('sample1=', A.sample(False, m/n).collect())
print('sample2=', A.sample(False, m/n).collect())

### Observații

- La fiecare execuție a celulei precedente, obținem o estimare diferită.
- Acuratețea estimării este determinată de dimensiunea eșantionului $n*p$. În acest caz, probabilitatea este $p=\frac{m}{n}$.
- Modificați valoarea lui $p$ și observați ce se obține.


## Statistici de bază


In [ ]:
print("Maximum: ", A.max())
print("Minimum: ", A.min())
print("Mean (average): ", A.mean())
print("Standard deviation: ", A.stdev())

In [ ]:
A.stats()

## Mapping

### Operația `map` cu functie _lambda_:


In [ ]:
B = A.map(lambda x: x*x)

In [ ]:
B.collect()

### Operația `map` cu funcție Python:


In [ ]:
def square_if_odd(x):
    if x % 2 == 1:
        return x*x
    else:
        return x

In [ ]:
A.map(square_if_odd).collect()

### Metoda `flatmap` returnează un RDD nou ce rezultă aplicând o funcție tuturor elementelor RDD-ului inițial și apoi aplatizând rezultatele.


In [ ]:
A.flatMap(lambda x: (x, x*x)).collect()

## Grupări și _binning_

### `groupby` returnează un RDD cu elemente grupate (iterabil) conform cu o operație (funcție) pe grupuri dată.


In [ ]:
result = A.groupBy(lambda x: x % 2).collect()
sorted([(x, sorted(y)) for (x, y) in result])

### Metoda `histogram` preia o listă de bins/buckets și returnează un tuplu ce conține rezultatul histogramei (binning).


In [ ]:
B.histogram([x for x in range(0, 100, 10)])

## Operații pe mulțimi

### Se vor crea RDD-uri mai mici pentru a ilustra aceste operații.


In [ ]:
lst1 = np.random.randint(0, 10, 3)
C = sc.parallelize(lst1)
lst2 = np.random.randint(10, 20, 3)
D = sc.parallelize(lst2)
print("C:", C.collect())
print("D:", D.collect())

### Operatia `C+D` reprezintă reuniunea (similară reuniunii de mulțimi), nu însumarea elementelor de pe aceleasi poziții.


In [ ]:
(C+D).collect()

### Metoda `cartesian` determină produsul cartezian (ca tupluri).


In [ ]:
C.cartesian(D).collect()

### Metodele `intersection` și `subtract` returnează un RDD reprezentând intersecția, respectiv diferența de mulțimi.


In [ ]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

In [ ]:
rdd1.subtract(rdd2).collect()

### Opriți `SparkContext`.


In [ ]:
sc.stop()